# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [74]:
# general + data loading
import pandas as pd
from sqlalchemy import create_engine
from itertools import chain

# pre-processing
from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.chunk import tree2conlltags
import re

# sklearn - general
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
# sklearn - classifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
# sklearn - metrics
from sklearn.metrics import f1_score, precision_score, recall_score
# sklearn - customs
from sklearn.base import BaseEstimator, TransformerMixin

# storage
import pickle
import time

In [1]:
# download relevant wordnet models
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('omw')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Felix\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-

True

In [3]:
# load data from database
engine = create_engine('sqlite:///../data/disaster_data.db')
df = pd.read_sql_table('texts', engine)
df.head()
X = df[['message', 'original', 'genre']]
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [29]:
# init these globally (to avoid call for every run)
stops = set(stopwords.words('english'))
wlem = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    '''Matches the given treebank POS tag to a wordnet one to be used by lemmatizer.'''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 'n'

def tokenize(text):
    '''Tokenizes the words in the text, uses lemmatization and POS tagging.'''
    # remove punctuation
    text = re.sub("[\.,\\:;!?'\"-]", " ", text.lower())
    tokens = word_tokenize(text)
    
    # pos tags and remove stopwords
    tags = pos_tag(tokens)
    tags = filter(lambda x: x[0] not in stops, tags)
    
    # part of speech
    tags = map(lambda x: (wlem.lemmatize(x[0], pos=get_wordnet_pos(x[1])), x[1]), tags)
    
    return list(tags)

def tokenize_clean(text):
    '''Removes POS Tags from the words.'''
    return list(map(lambda x: x[0], tokenize(text)))

def tokenize_ner(text):
    '''Applies Named Entity Recognition to the words.'''
    # remove punctuation
    text = re.sub("[\.,\\:;!?'\"-]", " ", text.lower())
    sents = sent_tokenize(text)
    
    tokens = chain.from_iterable(map(lambda x: tree2conlltags( ne_chunk(pos_tag(word_tokenize(x))) ), sents))
    tokens = filter(lambda x: x[0] not in stops, tokens)
    tokens = map(lambda x: (wlem.lemmatize(x[0], pos=get_wordnet_pos(x[1])), x[1], x[2]), tokens)
    
    return list(tokens)

def extract_ner(text):
    '''Extracts a list of Named Entities as additional feature.'''
    text = re.sub("[\.,\\:;!?'\"-]", " ", text.lower())
    sents = sent_tokenize(text)
    tokens = chain.from_iterable(map(lambda x: tree2conlltags( ne_chunk(pos_tag(word_tokenize(x))) ), sents))
    tokens = filter(lambda x: x[2] != "O", tokens)
    
    return list(tokens)
    
#%timeit X['message'].head(20).apply(tokenize_ner).head()
X['message'].apply(extract_ner).unique()

TypeError: unhashable type: 'list'

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [31]:
# simple baseline model
pipeline = [
    ('vectorize', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('cls', MultiOutputClassifier(LogisticRegression(), n_jobs=-1) )
]
pipeline = Pipeline(pipeline)

Define a function to add the test results to the csv file:

In [63]:
def _metric(y_test, y_pred, avg='binary'):
    '''Returns scores of the model.'''
    f1 = f1_score(y_test, y_pred, average=avg)
    prec = precision_score(y_test, y_pred, average=avg)
    rec = recall_score(y_test, y_pred, average=avg)
    return f1, prec, rec

def score_and_doc(model, name, X_test, y_test, extended=False):
    '''General scoring function.
    
    Takes the model and predicts output against given data.
    Finally scores them along different metrics and writes the results to the experiments file.
    
    Args:
        model: The sklearn model
        name (str): name of the model (used for documentation)
        X_test: test data
        y_test: expected test labels
    '''
    # predict the data
    y_pred = model.predict(X_test)
    
    # score the model (general)
    avg = 'micro'
    f1, prec, rec = _metric(y_test, y_pred, avg)
    
    # retrieve current config data?
    config = str(model.get_params())
    config = re.sub("[\n\t]", " ", config)
    config = re.sub("[,]", "/", config)
    config = re.sub("[ ]+", " ", config)
    
    # open file and append
    f=open("../experiments.csv", "a+")
    f.write("{},{:.6f},{:.6f},{:.6f},{},{}\n".format(name, f1, prec, rec, time.ctime(), config))
    f.close()
    
    # print output
    print("{}: F1 Score = {:.6f} (P={:.6f} / R={:.6f})".format(name, f1, prec, rec))
    
    # score the model (class-wise)
    if extended:
        # calculate the score for each cateogry
        cats = y_test.columns
        for i, c in enumerate(cats):
            sf1, sprec, srec = _metric(y_test.iloc[:, i], y_pred[:, i])
            print("  {:25} F1 Score = {:.6f} (P={:.6f} / R={:.6f})".format(c + ":", sf1, sprec, srec))
    
    

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# create small (mobile) dataset
df_s = df.sample(2000)
Xs_train, Xs_test, ys_train, ys_test = train_test_split(df_s[['message', 'original', 'genre']], df_s.iloc[:, 4:], test_size=0.3, random_state=42)

In [60]:
pipeline.fit(X_train['message'], y_train)

Pipeline(memory=None,
         steps=[('vectorize',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function token...
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('cls',
                 MultiOutputClassifier(estimator=LogisticRegression(C=1.0,
                                                                    class_weight=None,
                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [62]:
score_and_doc(pipeline, 'baseline', X_test['message'], y_test, extended=True)

baseline: F1 Score = 0.637465 (P=0.815869 / R=0.523084)
  related:                  F1 Score = 0.889864 (P=0.828105 / R=0.961577)
  request:                  F1 Score = 0.648598 (P=0.839178 / R=0.528561)
  offer:                    F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  aid_related:              F1 Score = 0.705550 (P=0.768368 / R=0.652227)
  medical_help:             F1 Score = 0.210667 (P=0.652893 / R=0.125596)
  medical_products:         F1 Score = 0.183007 (P=0.792453 / R=0.103448)
  search_and_rescue:        F1 Score = 0.029268 (P=1.000000 / R=0.014851)
  security:                 F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  military:                 F1 Score = 0.031128 (P=0.307692 / R=0.016393)
  water:                    F1 Score = 0.581064 (P=0.805755 / R=0.454361)
  food:                     F1 Score = 0.696317 (P=0.866782 / R=0.581882)
  shelter:                  F1 Score = 0.503401 (P=0.824841 / R=0.362238)
  clothing:                 F1 Score = 0.195489 (P=0.684

c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)



  missing_people:           F1 Score = 0.022472 (P=1.000000 / R=0.011364)
  refugees:                 F1 Score = 0.028269 (P=0.666667 / R=0.014440)
  death:                    F1 Score = 0.331858 (P=0.949367 / R=0.201072)
  other_aid:                F1 Score = 0.173267 (P=0.589888 / R=0.101547)
  infrastructure_related:   F1 Score = 0.050485 (P=0.619048 / R=0.026316)
  transport:                F1 Score = 0.118919 (P=0.916667 / R=0.063584)
  buildings:                F1 Score = 0.286902 (P=0.884615 / R=0.171216)
  electricity:              F1 Score = 0.190476 (P=0.857143 / R=0.107143)
  tools:                    F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  hospitals:                F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  shops:                    F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  aid_centers:              F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  other_infrastructure:     F1 Score = 0.005831 (P=0.111111 / R=0.002994)
  weather_related:          F1 Score 

### 6. Improve your model
Use grid search to find better parameters. 

In [181]:
parameters = {
    #'vectorize__tokenizer': (tokenize_clean, tokenize_ner),
    'vectorize__ngram_range': ((1, 1), (1, 2), (1, 4)),
    'vectorize__max_df': (0.5, 0.75, 1.0),
    'vectorize__max_features': (None, 5000, 10000),
    'tfidf__use_idf': (True, False),
    'cls__estimator__max_iter': [50, 100, 150],
    'cls__estimator__C': [0.5, 1.0, 2.0],
}

cv = GridSearchCV(pipeline, parameters, n_jobs=-1)
cv.fit(Xs_train['message'], ys_train)

c:\programming\envs\datascience\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vectorize',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                  

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [182]:
print("Best Parameters:")
print(cv.best_params_)
score_and_doc(cv, 'baseline_v2_s', Xs_test['message'], ys_test)

Best Parameters:
{'cls__estimator__max_iter': 50, 'cls__estimator__C': 2.0, 'tfidf__use_idf': False, 'vectorize__max_features': None, 'vectorize__max_df': 0.5, 'vectorize__ngram_range': (1, 1)}
baseline_v2_s: F1=0.604456 Prec=0.777409 Rec=0.494453


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

#### Input Feature Experiments

Various Experiments regarding different input features for the model. Those input features are tested separately and then in combined fashion.

In [65]:
# added Text Length as feature
def get_text_length(x):
    return np.array([len(t) for t in x]).reshape(-1, 1)

pipeline_length = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorize', CountVectorizer(tokenizer=tokenize_ner)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('length', Pipeline([
            ('count', FunctionTransformer(get_text_length, validate=False)),
        ]))
    ])),
    ('cls', MultiOutputClassifier(LogisticRegression(), n_jobs=-1) )
])
# fit and score
pipeline_length.fit(Xs_train['message'], ys_train)
score_and_doc(pipeline_length, 'feat_length_s', Xs_test['message'], ys_test, extended=True)

feat_length_s: F1 Score = 0.498741 (P=0.776036 / R=0.367444)
  related:                  F1 Score = 0.877767 (P=0.790295 / R=0.987013)
  request:                  F1 Score = 0.393701 (P=0.862069 / R=0.255102)
  offer:                    F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  aid_related:              F1 Score = 0.584475 (P=0.695652 / R=0.503937)
  medical_help:             F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  medical_products:         F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  search_and_rescue:        F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  security:                 F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  military:                 F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  water:                    F1 Score = 0.060606 (P=1.000000 / R=0.031250)
  food:                     F1 Score = 0.363636 (P=0.761905 / R=0.238806)
  shelter:                  F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  clothing:                 F1 Score = 0.000000 (P=

c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


In [66]:
# added Starting Verb
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = tokenize(sentence)
            if len(pos_tags) == 0: continue
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

pipeline_verb = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorize', CountVectorizer(tokenizer=tokenize_ner)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('starting_verb', StartingVerbExtractor())
    ])),
    ('cls', MultiOutputClassifier(LogisticRegression(), n_jobs=-1) )
])
# fit and score
pipeline_verb.fit(Xs_train['message'], ys_train)
score_and_doc(pipeline_verb, 'feat_verb_s', Xs_test['message'], ys_test)

feat_verb_s: F1 Score = 0.505376 (P=0.779867 / R=0.373807)


In [178]:
# use the genre as additional feature
pipeline_genre = Pipeline([
    ('features',
        ColumnTransformer(transformers=[
            ('text', Pipeline([
                ('vectorize', CountVectorizer(tokenizer=tokenize_ner)),
                ('tfidf', TfidfTransformer())
            ]), 'message'),
            ('genre', Pipeline([
                ('category', OneHotEncoder())
            ]), ['genre'])
        ]),
    ),
    ('cls', MultiOutputClassifier(LogisticRegression(), n_jobs=-1) )
])
# fit and score
pipeline_genre.fit(Xs_train, ys_train)
score_and_doc(pipeline_genre, 'feat_genre_s', Xs_test, ys_test)

feat_genre_s: F1=0.550017 Prec=0.767992 Rec=0.428420


**GloVe**

The next feature we are going to test is GloVe (Global Vectors for Word Representations). (GloVe is based on a factorization of co-occurence matrix to a lower dimensional space, while word2vec achieves a similar dimensionality reduction through a FF-NN).

We will use the pre-trained twitter vector-model here, under the assumption that it best captures the information in the message format of the datasets.

In [7]:
# define glove estimator
import sys
# updating path and load glove
sys.path.insert(1, '../models')
import glove

In [215]:
# reload glove (for dev)
import importlib
importlib.reload(glove)

# testing multiple params of glove - 
for dim, agg_type, max_feat in [(25, 'sent', None), (25, 'sent-matrix', 10), (25, 'sent-matrix', 20), (25, 'centroid', 5), (25, 'centroid', 15), (100, 'sent', None), (100, 'sent-matrix', 10)]:
    print("starting {} - {}".format(dim, agg_type))
    pipeline_glove = Pipeline([
        ('glove', glove.GloVeTransformer('twitter', dim, agg_type, tokenizer=tokenize_clean, max_feat=max_feat)),
        ('cls', MultiOutputClassifier(LogisticRegression(), n_jobs=-1) )
    ])

    pipeline_glove.fit(X_train['message'], y_train)
    score_and_doc(pipeline_glove, 'feat_glove_{}d_{}{}'.format(dim, agg_type, "" if max_feat is None else "_{}f".format(max_feat)), X_test['message'], y_test)

starting 100 - sent-matrix
feat_glove_100d_sent-matrix: F1=0.528374 Prec=0.598596 Rec=0.472897


Note that the 100Dim Embeddings perform worse than the 25Dim ones. It might be, that the LogisticRegression classifier cannot capture the information in such high dimensionalities, we will try a different classifier to support this (we will use SVMs, which should be suited for high dimensional data):

In [214]:
pipeline_glove = Pipeline([
    ('glove', glove.GloVeTransformer('twitter', 100, 'sent', tokenizer=tokenize_clean)),
    ('cls', MultiOutputClassifier(SVC(), n_jobs=-1) )
])

pipeline_glove.fit(X_train['message'], y_train)
score_and_doc(pipeline_glove, 'feat_glove_{}d_{}'.format(dim, agg_type), X_test['message'], y_test)

feat_glove_100d_sent: F1=0.367123 Prec=0.764399 Rec=0.241572


The last test to do, is the combination of glove with previous features

In [ ]:
# compose a feature union
pipeline_glove = Pipeline([
    ('features', FeatureUnion([
        ('term_emb', Pipeline([
            ('vectorize', CountVectorizer(tokenizer=tokenize_ner)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('glove', Pipeline([
            ('glove', glove.GloVeTransformer('twitter', 25, 'sent', tokenizer=tokenize_clean))
        ]))
    ])),
    ('cls', MultiOutputClassifier(LogisticRegression(), n_jobs=-1) )
])

pipeline_glove.fit(X_train['message'], y_train)
score_and_doc(pipeline_glove, 'feat_glove_tfidf_compose', X_test['message'], y_test)

**Putting it all together**

As we have a general understanding about the impact of the different features, lets try to combine the ones that delivered the best results into a single combined feature-set:

In [68]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, name):
        self.name = name
    
    def fit(self, x, y=None):
        return self
    
    def transform(self, X):
        return X[self.name]

# modularize the creation process
def create_pipeline_old(cls):
    '''Creates the feature pipeline with the specified classifier'''
    return Pipeline([
        ('features', FeatureUnion([
            ('text', Pipeline([
                ('msg', ColumnSelector('message')),
                ('embs', FeatureUnion([
                    ('term_emb', Pipeline([
                        ('vectorize', CountVectorizer(tokenizer=tokenize_ner, max_df=0.5)),
                        ('tfidf', TfidfTransformer(use_idf=False)),
                    ])),
                    ('glove', Pipeline([
                        ('glove_emb', glove.GloVeTransformer('twitter', 25, 'centroid', tokenizer=tokenize_clean, max_feat=5))
                    ]))
                ]))
            ])),
            ('genre', Pipeline([
                ('gnr', ColumnSelector(['genre'])),
                ('category', OneHotEncoder())
            ]))
        ])),
        ('cls',  cls)
    ])

def create_pipeline(cls):
    '''Creates the feature pipeline with the specified classifier'''
    return Pipeline([
        ('features', FeatureUnion([
            ('term_emb', Pipeline([
                ('vectorize', CountVectorizer(tokenizer=tokenize_ner, max_df=0.5)),
                ('tfidf', TfidfTransformer(use_idf=False)),
            ])),
            ('glove', Pipeline([
                ('glove_emb', glove.GloVeTransformer('twitter', 25, 'centroid', tokenizer=tokenize_clean, max_feat=5))
            ]))
        ])),
        ('cls',  cls)
    ])

# create the pipeline
pipeline_feat = create_pipeline( MultiOutputClassifier(LogisticRegression(), n_jobs=-1) )
# fit and score
pipeline_feat.fit(X_train['message'], y_train)
score_and_doc(pipeline_feat, 'feat_combined', X_test['message'], y_test, extended=True)

feat_combined: F1 Score = 0.652940 (P=0.780347 / R=0.561297)
  related:                  F1 Score = 0.891566 (P=0.856770 / R=0.929308)
  request:                  F1 Score = 0.656850 (P=0.793103 / R=0.560548)
  offer:                    F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  aid_related:              F1 Score = 0.701675 (P=0.737555 / R=0.669124)
  medical_help:             F1 Score = 0.327982 (P=0.588477 / R=0.227345)
  medical_products:         F1 Score = 0.291513 (P=0.580882 / R=0.194581)
  search_and_rescue:        F1 Score = 0.139130 (P=0.571429 / R=0.079208)
  security:                 F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  military:                 F1 Score = 0.333333 (P=0.470149 / R=0.258197)
  water:                    F1 Score = 0.644419 (P=0.744681 / R=0.567951)
  food:                     F1 Score = 0.715033 (P=0.817638 / R=0.635308)
  shelter:                  F1 Score = 0.564726 (P=0.745413 / R=0.454545)
  clothing:                 F1 Score = 0.310559 (P=

Perform an additional grid search over the parameters (focused on the weightings, etc. of the features):

In [ ]:
parameters = {
    #'vectorize__tokenizer': (tokenize_clean, tokenize_ner),
    'features__text__embs__term_emb__vectorize__ngram_range': ((1, 1), (1, 2)),
    'features__text__embs__term_emb__vectorize__max_df': (0.5, 0.75, 1.0),
    'features__text__embs__term_emb__vectorize__max_features': (None, 5000, 10000),
    'features__text__embs__term_emb__tfidf__use_idf': (True, False),
    'features__text__embs__glove__glove_emb__type': ('sent', 'sent-matrix', 'centroid'),
    'features__text__embs__glove__glove_emb__dim': (25, 100),
    'features__text__embs__glove__glove_emb__max_feat': (5, 10, 15),
    #'cls__estimator__max_iter': [50, 100, 150],
    #'cls__estimator__C': [0.5, 1.0, 2.0],
}

cv = GridSearchCV(pipeline_feat, parameters, n_jobs=-1)
cv.fit(Xs_train['message'], ys_train)

In [ ]:
print("Best Parameters:")
print(cv.best_params_)
score_and_doc(cv, 'feat_combined_v2', X_test[['message', 'genre']], y_test)

#### Classifier Experiments

Now that we have specified our features to use, lets experiment with various classifiers

In [80]:
def create_and_score(cls, name, small=False):
    '''Combines creation and scoring functions.
    
    Args:
        cls (Object): Classifier to be used in the pipeline
        name (str): Name for documentation of experiment
        small (bool): Defines if the small dataset should be used
        
    Returns:
        Trained Pipeline
    '''
    pipeline = create_pipeline(cls)
    if small:
        pipeline.fit(Xs_train['message'], ys_train)
        score_and_doc(pipeline, name + "_small", Xs_test['message'], ys_test, extended=True)
    else:
        pipeline.fit(X_train[['message', 'genre']], y_train)
        score_and_doc(pipeline, name, X_test['message'], y_test, extended=True)
    return pipeline

In [78]:
# decision tree
cls = MultiOutputClassifier(DecisionTreeClassifier(), n_jobs=-1)
pipeline_dt = create_and_score(cls, 'decision_tree', small=True)

# random forest
cls = MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1)
pipeline_rf = create_and_score(cls, 'random_forest', small=True)

decision_tree: F1 Score = 0.521477 (P=0.538722 / R=0.505302)
  related:                  F1 Score = 0.803047 (P=0.807440 / R=0.798701)
  request:                  F1 Score = 0.461538 (P=0.463918 / R=0.459184)
  offer:                    F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  aid_related:              F1 Score = 0.515856 (P=0.557078 / R=0.480315)
  medical_help:             F1 Score = 0.236842 (P=0.250000 / R=0.225000)
  medical_products:         F1 Score = 0.291667 (P=0.304348 / R=0.280000)
  search_and_rescue:        F1 Score = 0.214286 (P=0.214286 / R=0.214286)
  security:                 F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  military:                 F1 Score = 0.206897 (P=0.200000 / R=0.214286)
  water:                    F1 Score = 0.563380 (P=0.512821 / R=0.625000)
  food:                     F1 Score = 0.635659 (P=0.661290 / R=0.611940)
  shelter:                  F1 Score = 0.471545 (P=0.432836 / R=0.517857)
  clothing:                 F1 Score = 0.421053 (P=

c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)



  tools:                    F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  hospitals:                F1 Score = 0.142857 (P=0.111111 / R=0.200000)
  shops:                    F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  aid_centers:              F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  other_infrastructure:     F1 Score = 0.081633 (P=0.083333 / R=0.080000)
  weather_related:          F1 Score = 0.585366 (P=0.607595 / R=0.564706)
  floods:                   F1 Score = 0.377778 (P=0.377778 / R=0.377778)
  storm:                    F1 Score = 0.542373 (P=0.592593 / R=0.500000)
  fire:                     F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  earthquake:               F1 Score = 0.757895 (P=0.765957 / R=0.750000)
  cold:                     F1 Score = 0.181818 (P=0.400000 / R=0.117647)
  other_weather:            F1 Score = 0.107143 (P=0.150000 / R=0.083333)
  direct_report:            F1 Score = 0.440191 (P=0.438095 / R=0.442308)
random_forest: F1 Score = 0.437337 (P

c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)



  infrastructure_related:   F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  transport:                F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  buildings:                F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  electricity:              F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  tools:                    F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  hospitals:                F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  shops:                    F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  aid_centers:              F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  other_infrastructure:     F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  weather_related:          F1 Score = 0.303318 (P=0.780488 / R=0.188235)
  floods:                   F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  storm:                    F1 Score = 0.144928 (P=1.000000 / R=0.078125)
  fire:                     F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  earthquake:               F1 Score 

In [82]:
# Support Vector Classifier (might take some while)
cls = MultiOutputClassifier(SVC(C=0.5, degree=6), n_jobs=-1)
pipeline_sv = create_and_score(cls, 'support_vector', small=True)

support_vector_small: F1 Score = 0.371681 (P=0.770000 / R=0.244963)
  related:                  F1 Score = 0.870056 (P=0.770000 / R=1.000000)
  request:                  F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  offer:                    F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  aid_related:              F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  medical_help:             F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  medical_products:         F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  search_and_rescue:        F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  security:                 F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  military:                 F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  water:                    F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  food:                     F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  shelter:                  F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  clothing:                 F1 Score = 0.000

c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)



  hospitals:                F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  shops:                    F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  aid_centers:              F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  other_infrastructure:     F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  weather_related:          F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  floods:                   F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  storm:                    F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  fire:                     F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  earthquake:               F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  cold:                     F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  other_weather:            F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  direct_report:            F1 Score = 0.000000 (P=0.000000 / R=0.000000)


In [79]:
# Adaboost method (This might take a while...)
cls = MultiOutputClassifier(AdaBoostClassifier(base_estimator=RandomForestClassifier(), n_estimators=5), n_jobs=-1)
pipeline_sv = create_and_score(cls, 'adaboost_forest_classifier', small=True)

adaboost_forest_classifier: F1 Score = 0.436911 (P=0.754226 / R=0.307529)
  related:                  F1 Score = 0.872381 (P=0.778912 / R=0.991342)
  request:                  F1 Score = 0.097087 (P=1.000000 / R=0.051020)
  offer:                    F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  aid_related:              F1 Score = 0.451777 (P=0.635714 / R=0.350394)
  medical_help:             F1 Score = 0.095238 (P=1.000000 / R=0.050000)
  medical_products:         F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  search_and_rescue:        F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  security:                 F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  military:                 F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  water:                    F1 Score = 0.060606 (P=1.000000 / R=0.031250)
  food:                     F1 Score = 0.057971 (P=1.000000 / R=0.029851)
  shelter:                  F1 Score = 0.035088 (P=1.000000 / R=0.017857)
  clothing:                 F1 Score =

c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)



  other_aid:                F1 Score = 0.023810 (P=0.500000 / R=0.012195)
  infrastructure_related:   F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  transport:                F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  buildings:                F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  electricity:              F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  tools:                    F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  hospitals:                F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  shops:                    F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  aid_centers:              F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  other_infrastructure:     F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  weather_related:          F1 Score = 0.177083 (P=0.772727 / R=0.100000)
  floods:                   F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  storm:                    F1 Score = 0.030303 (P=0.500000 / R=0.015625)
  fire:                     F1 Score 

In [ ]:
parameters = {
    'cls__n_estimators': (2, 5, 10),
    'cls__criterion': ('gini', 'entropy'),
    'cls__max_depth': (None, 3, 5, 10, 15)
}

#cls = MultiOutputClassifier(DecisionTreeClassifier(), n_jobs=-1)
cls = MultiOutputClassifier(AdaBoostClassifer(base_estimator=DecisionTreeClassifier(), n_estimators=5), n_jobs=-1)
pipeline_sc = create_pipeline(cls)
#pipeline_sv = create_and_score(cls, 'adaboost_forest_classifier')

cv = GridSearchCV(pipeline_sv, parameters, n_jobs=-1)
cv.fit(Xs_train['message'], ys_train)

**Further Ideas:**

* Use Neural Networks for classification (a Feed-Forward Network on a Document Embedding or a RNN on word embeddings would be obvious choices)
* Create more second order features (e.g. search categories in the glove vector-space [e.g. through clustering] and create binary labels for these catgories (if at least one vector matches))
* Test against Word2Vec embeddings
* Apply more elaborate NLP Approches (such as BERT)
* More thorough data analysis and custom stop word removal
* Test XGBoosting approaches
* Balance Training Data (up/down sampling)
* Experiment with weights in the FeatureUnion

### 9. Export your model as a pickle file

In [83]:
# create the final model through external deps
import sys
sys.path.insert(1, '../models')
import glove
import ml_helper as utils

pipeline = Pipeline([
    ('features', FeatureUnion([
        ('term_emb', Pipeline([
            ('vectorize', CountVectorizer(tokenizer=utils.tokenize_clean, max_df=0.5)),
            ('tfidf', TfidfTransformer(use_idf=False)),
        ])),
        ('glove', Pipeline([
            ('glove_emb', glove.GloVeTransformer('twitter', 25, 'centroid', tokenizer=utils.tokenize_clean, max_feat=5))
        ]))
    ])),
    ('cls', MultiOutputClassifier(LogisticRegression(max_iter=50, C=2.0), n_jobs=-1) )
])
pipeline.fit(X_train['message'], y_train)
score_and_doc(pipeline, 'classify_final', X_test['message'], y_test, extended=True)

classify_final: F1 Score = 0.675088 (P=0.805797 / R=0.580866)
  related:                  F1 Score = 0.896006 (P=0.861862 / R=0.932967)
  request:                  F1 Score = 0.671681 (P=0.801478 / R=0.578065)
  offer:                    F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  aid_related:              F1 Score = 0.716186 (P=0.758502 / R=0.678341)
  medical_help:             F1 Score = 0.329466 (P=0.609442 / R=0.225755)
  medical_products:         F1 Score = 0.347985 (P=0.678571 / R=0.233990)
  search_and_rescue:        F1 Score = 0.237288 (P=0.823529 / R=0.138614)
  security:                 F1 Score = 0.013158 (P=1.000000 / R=0.006623)
  military:                 F1 Score = 0.310850 (P=0.546392 / R=0.217213)
  water:                    F1 Score = 0.655776 (P=0.771978 / R=0.569980)
  food:                     F1 Score = 0.742784 (P=0.829513 / R=0.672474)
  shelter:                  F1 Score = 0.615254 (P=0.780645 / R=0.507692)
  clothing:                 F1 Score = 0.374194 (P

c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\programming\envs\datascience\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


  hospitals:                F1 Score = 0.048780 (P=0.250000 / R=0.027027)
  shops:                    F1 Score = 0.000000 (P=0.000000 / R=0.000000)
  aid_centers:              F1 Score = 0.019608 (P=0.250000 / R=0.010204)
  other_infrastructure:     F1 Score = 0.064171 (P=0.300000 / R=0.035928)
  weather_related:          F1 Score = 0.773150 (P=0.845206 / R=0.712415)
  floods:                   F1 Score = 0.649376 (P=0.894180 / R=0.509804)
  storm:                    F1 Score = 0.621729 (P=0.745247 / R=0.533333)
  fire:                     F1 Score = 0.208333 (P=0.769231 / R=0.120482)
  earthquake:               F1 Score = 0.813559 (P=0.899023 / R=0.742934)
  cold:                     F1 Score = 0.301075 (P=0.717949 / R=0.190476)
  other_weather:            F1 Score = 0.202020 (P=0.537634 / R=0.124378)
  direct_report:            F1 Score = 0.568209 (P=0.733126 / R=0.463863)


In [70]:
# store the most promision pipeline
filename = '../models/pipeline.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.